In [3]:
import struct, socket
import shutil
import numpy as np
import pandas as pd
import linecache, bisect
import csv
import operator
import os
from IPython.html import widgets
from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display

spath =  'c:\\winpython\\notebooks\\oni-demo-win\\user\\vast\\'
sconnect = spath + 'lda_scores.csv'
sconnectbu = spath + 'lda_scores_bu.csv'
tmpconnect = sconnect+'.tmp'
stemp = sconnect + '.new'
coff = 150;
contxtpath = '\\oni-demo-win\\iploc\\'
nwloc = contxtpath + 'networkcontext.txt'
iploc = contxtpath + 'iploc.csv'

#print 'loading context data ... may take a few minutes.'
#iplist = np.loadtxt(iploc,dtype=np.uint32,delimiter=',',usecols={0},
#                    converters={0: lambda s: np.uint32(s.replace('"',''))})
srcdict,srclist = {},[]
dstdict,dstlist = {},[]
sportdict,sportlist = {},[]
dportdict,dportlist = {},[]
print('modules and data loaded')



modules and data loaded


C:\winpython\python-3.4.4.amd64\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
#shutil.copyfile(sconnectbu,sconnect)
#!cp $sconnectbu $sconnect

In [5]:
#load LDA scores#update lda doc with scores - either one edge,# or ALL that contain x IP and y port
def displaythis() :
    # build dict of IP addresses
    #sev,score, tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt
    srcdict,srclist = {},[]
    dstdict,dstlist = {},[]
    sportdict,sportlist = {},[]
    dportdict,dportlist = {},[]
    srclist.append('')
    dstlist.append('')
    sportlist.append('')
    dportlist.append('')
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        next(reader);
        rowct = 1
        for row in reader:
            if row[3] not in srcdict:
                srclist.append(row[3])
                srcdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
            if row[4] not in dstdict:
                dstlist.append(row[4])
                dstdict[row[4]] = struct.unpack("!L", socket.inet_aton(row[4]))[0]
            if row[5] not in sportdict:
                sportlist.append(row[5])
                sportdict[row[5]] = row[5]
            if row[6] not in dportdict:
                dportlist.append(row[6])
                dportdict[row[6]] = row[6]
            if rowct == coff:
                break;
            rowct += 1    
    searches = widgets.Box(width=550,height=200)
    searchp = widgets.HBox(width=550,height=150)
    actions = widgets.Box(width=50,height=150)
    srcselect = widgets.Select(options=srclist,height=min(len(srclist)*10,150),width=120)
    dstselect = widgets.Select(options=dstlist,height=min(len(dstlist)*10,150),width=120)
    sportselect = widgets.Select(options=sportlist,height=min(len(sportlist)*10,150),width=75)
    dportselect = widgets.Select(options=dportlist,height=min(len(dportlist)*10,150),width=75)
    tableHTML = ("<table width='100%'><tr><td>" +
    "<td>Source IP:</td><td>Dest IP:</td><td>Src Port</td><td>Dst Port:</td></tr></table>")
    srctext = widgets.Text(width=150, placeholder='Quick IP scoring')
    toplabel = widgets.HTML(value=tableHTML,width=550)
    ratingbut = widgets.RadioButtons(description='Rating:',options=['1', '2', '3'],width=50)
    assignbut = widgets.Button(description='Assign')
    updatebut = widgets.Button(description='Update')
    actions.children = [srctext,ratingbut,assignbut,updatebut]
    searchp.children = [srcselect,dstselect,sportselect,dportselect,actions]
    searches.children = [toplabel,searchp]
    display(searches)
    
    def update_sconnects(b):
        #define logic based on combo of input        
        if srctext.value !='':            
            src_t = ''
            dst_t = ''
            if srctext.value in srclist:
                src_t = srctext.value
            elif srctext.value in dstlist:
                dst_t= srctext.value
            svals = [src_t,dst_t,sportselect.value,dportselect.value]            
        else:
            svals = [srcselect.value,dstselect.value,sportselect.value,dportselect.value]
        risk = ratingbut.value
        shash = 0
        fhash = [3,4,5,6,11]
        for k in range(len(svals)):
            if svals[k] != '':                
                shash += 2**k
        with open(tmpconnect,'w') as g:
            riter = csv.writer(g,delimiter=',',lineterminator='\n')
            riter.writerow(["sev"]+["score"] + ["Date"] +["srcIP"]+["dstIP"]+["SPort"]+
                           ["DPort"]+["Sfirst"]+["Dfirst"]+["Stotal"]+["Dtotal"]+["SPkts"]+["Dpkts"]+["srcIpInternal"]+["destIpInternal"]
                           +["srcGeo"]+["dstGeo"]+["srcDomain"]+["dstDomain"]
                           +["gtiSrcRep"]+["gtiDstRep"]+["norseSrcRep"]+["norseDestRep"])
            with open(sconnect, 'r') as f:
                reader = csv.reader(f,delimiter=',')
                next(reader);
                rowct = 1
                #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
                for row in reader: 
                    result = 0
                    for n in range(0,5):
                        if (2**n & shash) > 0:                        
                            if row[fhash[n]] == svals[n]:
                                result += 2**n
                    if result == shash:
                        row[0] = risk
                        print(row)
                    riter.writerow(row) 
                    rowct += 1
        shutil.copyfile(tmpconnect,sconnect)
            
    def savesort(b):
        with open(stemp,'w') as g:
            riter = csv.writer(g,delimiter=',',lineterminator='\n')
            riter.writerow(["sev"]+["score"] + ["Date"] +["srcIP"]+["dstIP"]+["SPort"]+
                           ["DPort"]+["Sfirst"]+["Dfirst"]+["Stotal"]+["Dtotal"]+["SPkts"]+["Dpkts"]+["srcIpInternal"]+["destIpInternal"]
                           +["srcGeo"]+["dstGeo"]+["srcDomain"]+["dstDomain"]
                           +["gtiSrcRep"]+["gtiDstRep"]+["norseSrcRep"]+["norseDestRep"])
            reader = csv.reader(open(sconnect), delimiter=",")
            next(reader)
            srtlist = sorted(reader, key=lambda x: (int(x[0]), float(x[1])))
            riter.writerows(srtlist)
        #!mv $stemp $sconnect
        shutil.copyfile(stemp,sconnect)

    assignbut.on_click(update_sconnects)
    updatebut.on_click(savesort)


#bisect - linecache
# iterate through IP addresses and attach ip location info

# print IP addresses
def geoip() :
    srcdict,srclist = {},[]
    dstdict,dstlist = {},[]
    sportdict,sportlist = {},[]
    dportdict,dportlist = {},[]
    srclist.append('')
    dstlist.append('')
    sportlist.append('')
    dportlist.append('')
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        next(reader)
        rowct = 1
        for row in reader:
            if row[3] not in srcdict:
                srclist.append(row[3])
                srcdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
            if row[4] not in dstdict:
                dstlist.append(row[4])
                dstdict[row[4]] = struct.unpack("!L", socket.inet_aton(row[4]))[0]
            if row[5] not in sportdict:
                sportlist.append(row[5])
                sportdict[row[5]] = row[5]
            if row[6] not in dportdict:
                dportlist.append(row[6])
                dportdict[row[6]] = row[6]
            if rowct == coff:
                break;
            rowct += 1    
            linecache.clearcache()
    for srcip in srcdict:
        srcdict[srcip] = linecache.getline(iploc, bisect.bisect(iplist,srcdict[srcip])).replace('\n','')
    for dstip in dstdict:
        dstdict[dstip] = linecache.getline(iploc, bisect.bisect(iplist,dstdict[dstip])).replace('\n','')
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        next(reader)
        rowct = 1
        for row in reader:
            #print "line",rowct,row,":"
            if srcdict[row[3]] == '':
                srcip = 'unknown'
            else:
                srcip = srcdict[row[3]]
            if dstdict[row[4]] == '':
                dstip = 'unknown'
            else:
                dstip = dstdict[row[4]]
            srcgeoip = srcip.replace('"','').split(',')
            dstgeoip = dstip.replace('"','').split(',')
            
            print('row',rowct,':',row[3],srcgeoip[4:6],srcgeoip[8:10],"-->",row[4],dstgeoip[4:6],dstgeoip[8:10] )
            if rowct == 100:
                break
            rowct += 1

def set_rules():
    rops = ['leq','leq','leq','leq','leq','leq']
    rvals = ['','','',1024,'',54]
    risk = 2
    apply_rules(rops,rvals,risk)
    rops = ['leq','leq','leq','leq','eq','eq']
    rvals = ['','','',1024,3,152]
    risk = 2
    apply_rules(rops,rvals,risk)
    
    rops = ['leq','leq','leq','leq','eq','eq']
    rvals = ['','','',1024,2,104]
    risk = 2
    apply_rules(rops,rvals,risk)
    
    rops = ['leq','leq','eq','leq','leq','leq']
    rvals = ['','',0,1023,'','']
    risk = 2

    #apply_rules(rops,rvals,risk)

def apply_rules(rops,rvals,risk):
    #define logic based on combo of input
    rhash = 0 # rule hash
    rfhash = [3,4,5,6,9,11] # Columns to compare with
    for k in range(len(rvals)):
        if rvals[k] != '':                
            rhash += 2**k
    with open(tmpconnect,'w') as g:
        riter = csv.writer(g,delimiter=',',lineterminator='\n')
        riter.writerow(["sev"]+["score"] + ["Date"] +["srcIP"]+["dstIP"]+["SPort"]+
                           ["DPort"]+["Sfirst"]+["Dfirst"]+["Stotal"]+["Dtotal"]+["SPkts"]+["Dpkts"]+["srcIpInternal"]+["destIpInternal"]
                           +["srcGeo"]+["dstGeo"]+["srcDomain"]+["dstDomain"]
                           +["gtiSrcRep"]+["gtiDstRep"]+["norseSrcRep"]+["norseDestRep"])
        with open(sconnect, 'r') as f:
            reader = csv.reader(f,delimiter=',')
            next(reader)
            rowct = 1
            #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
            for row in reader: 
                result = 0
                for n in range(0,6):
                    if (2**n & rhash) > 0:
                        if rops[n] == 'leq':
                            if int(row[rfhash[n]]) <= int(rvals[n]):
                                result += 2**n                           
                        if rops[n] == 'eq':
                            if int(row[rfhash[n]]) == int(rvals[n]):
                                result += 2**n                           
                if result == rhash:
                    row[0] = risk
                    print(row)
                    rowct += 1
                riter.writerow(row) 
    print('rows applied', rowct)         
    shutil.copyfile(tmpconnect,sconnect)

def attack_heuristics():
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        next(reader)
        rowct = 1
        for row in reader:
            if row[3] not in srcdict:
                srcdict[row[3]] = row[3]
            if row[4] not in dstdict:
                 dstdict[row[4]] = row[4]
            if row[5] not in sportdict:
                sportdict[row[5]] = row[5]
            if row[6] not in dportdict:
                dportdict[row[6]] = row[6]


    df = pd.read_csv(sconnect, error_bad_lines=False)
    
   
    #print df

    gb = df.groupby([u'srcIP'])
    
    #print gb.groups.keys()
    
    for srcip in gb.groups.keys(): #srcdict:
        if len(gb.get_group(srcip)) > 20:
            print(srcip,'connects:',len(gb.get_group(srcip)) )


In [6]:
#shutil.copy(sconnectbu,sconnect)

In [7]:
attack_heuristics()

172.30.0.70 connects: 24
10.138.235.111 connects: 24
10.13.77.49 connects: 25


In [8]:
displaythis()
#geoip()

In [9]:
set_rules()

[2, '0.000000149', '9/14/2015 17:16', '10.13.77.49', '172.10.0.40', '47131', '80', '0', '0', '206', '162', '3', '3', '0', '1', 'CountryD;CityA', 'Texas;Richardson Laurie Hatfield DBA', 'countryd.citya.com', 'sbcglobal.net', '0', '', 'Medium', '']
[2, '0.000000149', '9/14/2015 14:56', '10.13.77.49', '172.10.0.3', '35579', '25', '0', '0', '112', '58', '2', '1', '0', '1', 'CountryD;CityA', 'California;San Diego Gregory Hout Attorney at Law', 'countryd.citya.com', 'sbcglobal.net', '0', '', 'Medium', '']
[2, '0.00000015', '9/14/2015 15:10', '10.70.68.127', '172.30.0.4', '6395', '80', '0', '0', '278', '116', '5', '2', '0', '1', 'CountryN;CityA', '-;- Private IP Address LAN', 'countryn.citya.com', '-', '22', '', 'Low', '']
[2, '0.00000015', '9/14/2015 15:09', '10.70.68.127', '172.30.0.4', '55759', '80', '19', '1006', '297', '1114', '5', '2', '0', '1', 'CountryN;CityA', '-;- Private IP Address LAN', 'countryn.citya.com', '-', '22', '', 'Low', '']
[2, '0.000000151', '9/10/2015 17:16', '10.13.77